# Image generator
This notebook is responsible for generating labeled images from the matlab files.
Each image is labeled as one of 'fn', 'fy', 'mn', 'my'. These labels are then split up into two binary labels during training.

the structure of the matlab files is as follows
```
cells[11]
    subjects[13]
        channels[sampels;2]
```

Each cell represents a different set of channels. The channels used by each is:
1. Cz, F4
2. Cz, F4
3. Cz, F4
4. Cz, F4
5. Cz, F4
6. Cz, F4
7. 01, F4
8. 01, F4
9. 01, F4
10. F3, F4
11. Fz F4

the training is sepparated by what channels are in use.


In [124]:
from scipy.io.matlab import loadmat
from torch.fft import fft2
from torch import tensor, zeros
from torchvision.io import write_png
from os import makedirs
import matplotlib.pyplot as plt

In [125]:
cell_channels = [
    ['Cz', 'F4'],
    ['Cz', 'F4'],
    ['Cz', 'F4'],
    ['Cz', 'F4'],
    ['Cz', 'F4'],
    ['Cz', 'F4'],
    ['01', 'F4'],
    ['01', 'F4'],
    ['01', 'F4'],
    ['F3', 'F4'],
    ['Fz', 'F4'],
]

cell_maps = {
    'Cz': 0,
    '01': 2,
    'F3': 4,
    'Fz': 6,
    'F4': 8
}

In [126]:


def load_file(name: str):
    file = loadmat(f'data/EEG/{name}.mat')
    cells = file[name]
    gender = 'm' if name.startswith('M') else 'f'
    adhd = 'y' if not name.endswith('C') else 'n'
    dir = f'data/generated/{gender}{adhd}'
    makedirs(dir, exist_ok=True)
    for cell in range(cells.shape[1]):
        sample = cells[0,cell]
        for p in range(sample.shape[0]):
            # subject 7 is corrupt amongs the women
            if p == 7 and name == 'FADHD':
                continue
            person = sample[p]
            sampled = fft2(tensor(person))

            index = 0
            while index < sampled.shape[0]:
                start = index
                end = index+100
                index += 100
                if index > sampled.shape[0]:
                    start = sampled.shape[0]-100
                    end = sampled.shape[0]

                image = zeros((1, 100, 4))
                [cell_a, cell_b] = cell_channels[cell]

                image[0, :,0] = sampled[start:end,0].real
                image[0, :,1] = sampled[start:end,0].imag
                image[0, :,2] = sampled[start:end,1].real
                image[0, :,3] = sampled[start:end,1].imag

                image = image * 255
                image = image.byte()
                
                filename = f'{dir}/c{cell}p{p}s{index//100}d{cell_a}{cell_b}.png'
                write_png(image, filename)


In [127]:
load_file('FC')
load_file('MC')
load_file('MADHD')
load_file('FADHD')